### Fisher score

In [1]:
import numpy as np

def fisher_score(X, y):
    """
    Fisher Score for each feature.
        X: Feature matrix (N samples x M features)
        y: Labels (N samples)
    Returns:
        scores: Fisher Scores for all features (array of length M)
    """
    n_samples, n_features = X.shape
    unique_classes = np.unique(y)
    scores = np.zeros(n_features)
    
    for i in range(n_features):
        overall_mean = np.mean(X[:, i])  # Mean of the feature across all samples
        between_class_var = 0
        within_class_var = 0
        
        for cls in unique_classes:
            class_indices = np.where(y == cls)[0]
            class_data = X[class_indices, i]
            class_mean = np.mean(class_data)
            class_var = np.var(class_data)
            
            between_class_var += len(class_data) * (class_mean - overall_mean) ** 2
            within_class_var += len(class_data) * class_var
            
        # Calculate Fisher Score for the feature
        scores[i] = between_class_var / (within_class_var + 1e-6)  # Avoid division by zero
    
    return scores


### ReliefF

In [2]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

def reliefF(X, y, k=10):
    """
    ReliefF feature ranking.
    Args:
        X: Feature matrix (N samples x M features)
        y: Labels (N samples)
        k: Number of nearest neighbors (default is 10)
    Returns:
        scores: Feature importance scores (higher is better)
    """
    n_samples, n_features = X.shape
    scores = np.zeros(n_features)
    
    nn = NearestNeighbors(n_neighbors=k+1)  # +1 to exclude the sample itself
    nn.fit(X)

    for i in range(n_samples):
        # Find the k nearest neighbors (including the sample itself)
        distances, indices = nn.kneighbors([X[i]])
        
        # Exclude the sample itself (1st index is the sample itself)
        indices = indices[0][1:]
        
        # Find nearest hits and nearest misses
        near_hits = indices[y[indices] == y[i]]
        near_misses = indices[y[indices] != y[i]]
        
        # Update scores for each feature
        for feature in range(n_features):
            hit_diff = np.sum((X[i, feature] - X[near_hits, feature]) ** 2)
            miss_diff = np.sum((X[i, feature] - X[near_misses, feature]) ** 2)
            scores[feature] += hit_diff - miss_diff

    # Normalize scores by the number of samples
    scores /= n_samples
    
    return scores


### Robust multi-label feature selection with dual-graph regularization:

In [14]:
def drmfs(X, Y, lambda1=0.1, lambda2=0.1, lambda3=0.1):
    N, M = X.shape
    N, L = Y.shape
    dist_X = pairwise_distances(X, metric='cosine')
    dist_Y = pairwise_distances(Y, metric='jaccard')
    L_X = np.exp(-dist_X**2 / (2 * 1.0 ** 2))
    np.fill_diagonal(L_X, 0)
    L_Y = np.exp(-dist_Y**2 / (2 * 1.0 ** 2))
    np.fill_diagonal(L_Y, 0)
    W = cp.Variable((M, L))
    data_fitting = cp.norm(Y - X @ W, 'fro')**2
    feature_sparsity = lambda1 * cp.norm(W, 'fro')**2
    feature_graph = lambda2 * cp.trace(W.T @ L_X @ W)
    label_graph = lambda3 * cp.trace(W.T @ L_Y @ W)
    objective = cp.Minimize(data_fitting + feature_sparsity + feature_graph + label_graph)
    problem = cp.Problem(objective)
    problem.solve()
    return W.value


In [5]:
import numpy as np

def vikor(decision_matrix, weights, v=0.5):
    # Step 1: Identify f* and f-
    f_star = np.max(decision_matrix, axis=0)
    f_minus = np.min(decision_matrix, axis=0)
    
    # Step 2: Compute S and R
    S = np.zeros(decision_matrix.shape[0])
    R = np.zeros(decision_matrix.shape[0])
    for i in range(decision_matrix.shape[0]):
        normalized_diff = weights * (f_star - decision_matrix[i]) / (f_star - f_minus)
        S[i] = np.sum(normalized_diff)
        R[i] = np.max(normalized_diff)
    
    # Step 3: Compute Q
    S_star, S_minus = np.min(S), np.max(S)
    R_star, R_minus = np.min(R), np.max(R)
    Q = np.zeros(decision_matrix.shape[0])
    for i in range(decision_matrix.shape[0]):
        Q[i] = v * (S[i] - S_star) / (S_minus - S_star) + (1 - v) * (R[i] - R_star) / (R_minus - R_star)
    
    # Step 4: Rank features
    rankings = np.argsort(Q)  # Ascending order
    return Q, rankings


In [6]:
import numpy as np

def topsis(decision_matrix, weights):
    """
    Implements TOPSIS for ranking features.
    Args:
        decision_matrix: 2D array (features x criteria).
        weights: 1D array of criteria weights (must sum to 1).
    Returns:
        closeness: Array of relative closeness scores (C_i).
        rankings: Array of feature indices sorted by rank (descending).
    """
    # Step 1: Normalize the decision matrix
    norm_matrix = decision_matrix / np.sqrt((decision_matrix ** 2).sum(axis=0))
    
    # Step 2: Apply weights
    weighted_matrix = norm_matrix * weights
    
    # Step 3: Determine ideal and negative-ideal solutions
    ideal_solution = np.max(weighted_matrix, axis=0)
    negative_ideal_solution = np.min(weighted_matrix, axis=0)
    
    # Step 4: Compute distances
    distance_to_ideal = np.sqrt(((weighted_matrix - ideal_solution) ** 2).sum(axis=1))
    distance_to_negative_ideal = np.sqrt(((weighted_matrix - negative_ideal_solution) ** 2).sum(axis=1))
    
    # Step 5: Calculate relative closeness
    closeness = distance_to_negative_ideal / (distance_to_ideal + distance_to_negative_ideal)
    
    # Step 6: Rank features
    rankings = np.argsort(closeness)[::-1]
    
    return closeness, rankings


In [7]:
import numpy as np

def codas(decision_matrix, weights, alpha=0.5):
    # Step 1: Normalize the decision matrix
    norm_decision_matrix = decision_matrix / np.sqrt(np.sum(decision_matrix**2, axis=0))

    # Step 2: Calculate the weighted normalized decision matrix
    weighted_matrix = norm_decision_matrix * weights

    # Step 3: Calculate the negative ideal solution
    negative_ideal_solution = np.min(weighted_matrix, axis=0)

    # Step 4: Calculate Euclidean and Taxicab distances
    euclidean_distances = np.sqrt(np.sum((weighted_matrix - negative_ideal_solution)**2, axis=1))
    taxicab_distances = np.sum(np.abs(weighted_matrix - negative_ideal_solution), axis=1)

    # Step 5: Calculate the assessment scores
    assessment_scores = euclidean_distances + alpha * taxicab_distances

    # Step 6: Rank the alternatives
    rankings = np.argsort(assessment_scores)

    return assessment_scores, rankings
